
> **AutoGen > User Guide > GroupChat > Customize Speaker Selection**
https://microsoft.github.io/autogen/0.2/docs/topics/groupchat/customized_speaker_selection

## Customize Speaker Selection

![](https://velog.velcdn.com/images/heyggun/post/e40a770a-6225-4e6a-b020-21118d2bb9b5/image.png)

GroupChat에서는 `GroupChat` 객체에 함수를 전달하여 **발화자 선택을 커스터마이징**할 수 있다. 이 함수를 사용하면 더 결정론적인 에이전트 워크플로우를 구성할 수 있으며, 이를 구현할 때는 **StateFlow 패턴**을 따르는 것을 권장한다. 자세한 내용은 StateFlow 블로그를 참고

> **StateFlow**
https://microsoft.github.io/autogen/0.2/blog/2024/02/29/StateFlow/
**StateFlow 정리 포스팅**
https://velog.io/@heyggun/AutoGen-StateFlow-GroupChat%EC%97%90%EC%84%9C-%EC%82%AC%EC%9A%A9%EC%9E%90-%EC%A7%80%EC%A0%95-%EB%B0%9C%ED%99%94%EC%9E%90-%EC%84%A0%ED%83%9D%EC%9D%B4-%EA%B0%80%EB%8A%A5%ED%95%9C-%EC%83%81%ED%83%9C-%EA%B8%B0%EB%B0%98-%EC%9B%8C%ED%81%AC%ED%94%8C%EB%A1%9C%EC%9A%B0-%EA%B5%AC%EC%B6%95

### An example research workflow

다음은 사용자 지정 발화자 선택 기능이 포함된 연구용 **StateFlow** 모델을 구축하는 간단한 예시이다. 먼저 다음과 같은 에이전트들을 정의한다.

- **Initializer** (초기화자) : 작업을 전송하여 워크플로우를 시작한다.
- **Coder** (코더) : 코드를 작성하여 인터넷에서 논문을 검색한다.
- **Executor** (실행자) : 코드를 실행한다.
- **Scientist** (과학자) : 논문을 읽고 요약을 작성한다.

위 그림에서는 4개의 상태(`Init`, `Retrieve`, `Research`, `End`)로 구성된 간단한 연구 워크플로우를 정의한다. 각 상태에서는 다양한 에이전트들을 호출하여 작업을 수행한다.

- **Init** (초기화) : *Initializer*를 사용하여 워크플로우를 시작한다.
- **Retrieve** (검색) :  먼저 *Coder*를 호출하여 코드를 작성하고, 이후 *Executor*를 호출하여 코드를 실행한다.
- **Research** (연구) : *Scientist*를 호출하여 논문을 읽고 요약을 작성하게 한다.
- **End** (종료) : 워크플로우를 종료한다.


### Create your speaker selection function

다음은 `speaker selection` 함수의 뼈대이다. 이 함수에 `speaker selection` 로직을 정의하여 완성한다. 

In [6]:
import autogen
from autogen import Agent, GroupChat
from typing import Literal, Union

In [8]:
def custom_speaker_selection_func(
    last_speaker: Agent,
    groupchat: GroupChat
) -> Union[Agent, Literal["auto", "manual", "random", "round_robin"], None]:
    
    """
    Define a customized speaker selection function.
    A recommended way is to define a transition for each speaker in the groupchat.
    
    Parameters:
        - last_speaker : Agent
            The last speaker in the group chat.
        - groupchat : GroupChat
            The GroupChat object

    Return:
        Return one of the following:
        1. an `Agent` class, it must be one of the agents in the group chat.
        2. a string from ["auto", "manual", "random", "round_roubin"] to select a default method to use.
        3. None, which indicates the chat should be terminated.
    """
    pass
 

In [9]:
from config import settings

api_key = settings.openai_api_key.get_secret_value()

In [15]:
llm_config = {
    "config_list":
        [
            {
                "model" : "gpt-4o-mini",
                "api_key" : api_key
            }
        ]
}

code_execution_config= {
    "last_n_messages" : 3,
    "work_dir" : "paper",
    "use_docker" : False,
}

In [14]:
from autogen import AssistantAgent, UserProxyAgent, ConversableAgent


In [17]:
initializer = UserProxyAgent(
    name="init"
)

coder = autogen.AssistantAgent(
    name="Retrieve_Action_1",
    llm_config = llm_config,
    system_message= """You are the Coder. Given a topic, write code to retrieve related papers form the arXiv API, print their title, authors, abstract, and link.
You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify.
Don't use a code block if it's not intended to be executed by the executor. 
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result.
Check the execution result return ed by the executor.
If the result indicates there is an error, fix the error and output the code again.
Suggest the full code instread of partial code or code changes.
If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit you assumption, collect additional info you need, and think of a different approach to try.
"""
)

executor = autogen.UserProxyAgent(
    name="Retrieve_Action_2",
    system_message= "Executor. Execute the code written by the Coder and report the reulst.",
    human_input_mode="NEVER",
    code_execution_config=code_execution_config,
)

scientist = autogen.AssistantAgent(
    name="Research_Action_1",
    llm_config=llm_config,
    system_message="""You are the Scientist. Please categorize papers after seeing their abstracts printed and create a markdown table with Domain, Titel, Authors, Summary and Link"""
)

In [18]:
def state_transition(last_speaker, groupchat):
    messages = groupchat.messages
    
    if last_speaker is initializer:
        return coder
        
    elif last_speaker is coder:
        return executor
    
    elif last_speaker is executor:
        if messages[-1]["content"] == "exitcode: 1":
            return coder
        
        else:
            return scientist
        
    elif last_speaker == "Scientist":
        return None
    

In [19]:
groupchat = autogen.GroupChat(
    agents=[initializer, coder, executor, scientist],
    messages=[],
    max_round=20,
    speaker_selection_method=state_transition,
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [20]:
result = initializer.initiate_chat(
    manager,
    message="Topic : LLM applications papers from last week. Requirement : 5- 10 papers from different domains."
)

init (to chat_manager):

Topic : LLM applications papers from last week. Requirement : 5- 10 papers from different domains.

--------------------------------------------------------------------------------

Next speaker: Retrieve_Action_1

Retrieve_Action_1 (to chat_manager):

```python
import requests
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET

def get_arxiv_papers(topic, days=7, max_results=10):
    # Calculate the date from one week ago
    end_date = datetime.utcnow()
    start_date = end_date - timedelta(days=days)

    # Format dates for arXiv's API query
    start_date_str = start_date.strftime('%Y%m%d')
    end_date_str = end_date.strftime('%Y%m%d')

    # Define the arXiv API URL
    url = f'http://export.arxiv.org/api/query?search_query=all:{topic}&start=0&max_results={max_results}&sortBy=submittedDate&sortOrder=descending'
    
    response = requests.get(url)
    
    if response.status_code == 200:
        # Parse the XML response
        r

In [26]:
from pprint import pprint

In [27]:
pprint(result.chat_history[-1]['content'])

('Here is the markdown table summarizing the LLM applications papers retrieved '
 'from the last week:\n'
 '\n'
 '| Domain               | '
 'Title                                                                                                                   '
 '| '
 'Authors                                                                                                                   '
 '| '
 'Summary                                                                                                                                                                                                                                                                                                                                                                                                                                         '
 '| Link                                       |\n'
 '|----------------------|-----------------------------------------------------------------------------------